In [1]:
import pandas as pd
import sys
from pyspark.sql import types as T
from pyspark.sql import functions as F

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas
import math
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [17]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Test_spark").master("local[*]").getOrCreate()
spark

In [5]:
# Importamos los datos de entrenamiento y los de prueba
ruta="C:/Users/Aitor/Desktop/TFM/"
fichero_entrenamiento = "entrenamiento_5_int_final.csv"
fichero_prueba = "prueba_1_int_final.csv"

In [7]:
# Random seed for reproducibility
seed = 10
np.random.seed(seed)
# Import data
df_train = pd.read_csv(ruta+fichero_entrenamiento, header=0)
df_prueba = pd.read_csv(ruta+fichero_prueba, header=0)
# Print first 10 samples
print(df_prueba.head(5))

   user_ID  attemp   accX(g)  gyrz(o/s)
0        1      10  1.011230  -0.916031
1        1      10  1.008301  -0.778626
2        1      10  1.001953  -0.824427
3        1      10  1.000000  -0.946565
4        1      10  1.005859  -0.870229


In [10]:
# Divide data into features X and target (Classes) Y
X_train = df_train.iloc[:,0:4]
Y_train = df_train.iloc[:,0]
X_prueba = df_prueba.iloc[:,0:4]
Y_prueba = df_prueba.iloc[:,0]
print(X_train)

        user_ID  attemp   accX(g)  gyrz(o/s)
0             1       1  1.009766  -0.778626
1             1       1  1.009277  -0.580153
2             1       1  1.006348  -0.625954
3             1       1  1.014648  -0.625954
4             1       1  1.015137  -0.916031
...         ...     ...       ...        ...
351320       47       5  1.012695  -0.396947
351321       47       5  1.007812  -0.763359
351322       47       5  1.013672  -0.671756
351323       47       5  0.998535  -0.809160
351324       47       5  0.997559  -0.503817

[351325 rows x 4 columns]


In [11]:
# Normalize features within range 0 (minimum) and 1 (maximum)
scaler = MinMaxScaler(feature_range=(0, 1))
X_train = scaler.fit_transform(X_train)
X_train = pd.DataFrame(X_train)

In [13]:
# Convert target Y to one hot encoded Y for Neural Network
Y = pd.get_dummies(Y_train)
# If target is in string form, use following code:
# First encode target values as integers from string
# Then perform one hot encoding
# encoder = LabelEncoder()
# encoder.fit(Y)
# Y = encoder.transform(Y)
# Y = np_utils.to_categorical(Y)

In [21]:
# For Keras, convert dataframe to array values (Inbuilt requirement of Keras)
X_train = X_train.values
Y_train = Y_train.values
X_prueba = X_prueba.values
Y_prueba = Y_prueba.values

In [22]:
# First define baseline model. Then use it in Keras Classifier for the training
def baseline_model():
    # Create model here
    model = Sequential()
    model.add(Dense(64, activation = 'relu')) # Rectified Linear Unit Activation Function
    model.add(Dense(64, activation = 'relu'))
    model.add(Dense(47, activation = 'softmax')) # Softmax for multi-class classification
    # Compile model here
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [23]:
# Create Keras Classifier and use predefined baseline model
estimator = KerasClassifier(build_fn = baseline_model, epochs = 8, batch_size = 512, verbose = 0)
# Try different values for epoch and batch size

In [24]:
# KFold Cross Validation
kfold = KFold(n_splits = 10, shuffle = True, random_state = seed)

In [25]:
# Object to describe the result
results = cross_val_score(estimator, X, Y, cv = kfold)
# Result
print("Result: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Result: 65.95% (2.07%)
